In [7]:
import torch
import os

device = 'cuda' if torch.cuda.is_available() else 'cpu'

baseDir = './dataset/cats_and_dogs_filtered'

trainDir = os.path.join(baseDir, 'train')
validationDir = os.path.join(baseDir, 'validation')


# 훈련 데이터 이미지
trainCatsDir = os.path.join(trainDir, 'cats')
trainDogsDir = os.path.join(trainDir, 'dogs')

print(trainCatsDir)
print(trainDogsDir)

# 테스트에 사용되는 경로
validationCatsDir = os.path.join(validationDir, 'cats')
validationDogsDir = os.path.join(validationDir, 'dogs')

print(validationCatsDir)
print(validationDogsDir)



./dataset/cats_and_dogs_filtered\train\cats
./dataset/cats_and_dogs_filtered\train\dogs
./dataset/cats_and_dogs_filtered\validation\cats
./dataset/cats_and_dogs_filtered\validation\dogs


학습을 위한 데이터 경로 설정 완료